In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.text import *

In [2]:
######sample dataset


path = untar_data(URLs.IMDB_SAMPLE)
path.ls()

[WindowsPath('C:/Users/91958/.fastai/data/imdb_sample/texts.csv')]

In [5]:
df = pd.read_csv(path /'texts.csv')

df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [6]:
data_lm = TextDataBunch.from_csv(path, 'texts.csv')

In [8]:
data_lm.show_batch()

text,target
"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n \n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , steaming bowl of xxunk . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj",negative
"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the sweetest and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with",positive
"xxbos xxmaj now that xxmaj che(2008 ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj sydney , after xxunk ) , i can xxunk join both xxunk of "" xxmaj at xxmaj the xxmaj movies "" in taking xxmaj steven xxmaj soderbergh to task . \n \n xxmaj it 's usually satisfying to watch a film director change his style /",negative
"xxbos xxmaj this film sat on my xxmaj xxunk for weeks before i watched it . i dreaded a self - indulgent xxunk flick about relationships gone bad . i was wrong ; this was an xxunk xxunk into the screwed - up xxunk of xxmaj new xxmaj xxunk . \n \n xxmaj the format is the same as xxmaj max xxmaj xxunk ' "" xxmaj la xxmaj ronde",positive
"xxbos xxmaj many neglect that this is n't just a classic due to the fact that it 's the first xxup 3d game , or even the first xxunk - up . xxmaj it 's also one of the first xxunk games , one of the xxunk definitely the first ) truly claustrophobic games , and just a pretty well - xxunk xxunk experience in general . xxmaj with graphics",positive


In [ ]:
data = (TextList.from_csv(path, 'texts.csv', cols='text')
                .split_from_df(col=2)
                .label_from_df(cols=0)
                .databunch())

####now download the full dataset--------------

In [9]:
path = untar_data(URLs.IMDB)

In [10]:
data_lm = (TextList.from_folder(path)
           #Inputs: all the text files in path
            .filter_by_folder(include=['train', 'test', 'unsup']) 
           #We may have other temp folders that contain text files so we only keep what's in train and test
            .split_by_rand_pct(0.1)
           #We randomly split and keep 10% (10,000 reviews) for validation
            .label_for_lm()           
           #We want to do a language model so we label accordingly
            .databunch(bs=50))

data_lm.show_batch()

idx,text
0,"later , by which time i did not care . xxmaj the character we should really care about is a very cocky , overconfident xxmaj ashton xxmaj kutcher . xxmaj the problem is he comes off as kid who thinks he 's better than anyone else around him and shows no signs of a cluttered closet . xxmaj his only obstacle appears to be winning over xxmaj costner . xxmaj"
1,"to survive and protect xxmaj nadia . \n \n xxmaj the career of xxmaj wesley xxmaj snipes is downhill . i have just seen this flick , and it is another disappointing movie of this actor , whose career is presently very similar to xxmaj steven xxmaj segal 's one . xxmaj the movie has many explosions , shots and car chase associated to an awful story and horrible"
2,", hands down , one of the weirdest i 've ever seen . xxmaj certainly one of the weirdest shlock films . xxmaj basically , a demon took human form years ago for a woman , the woman died or something , the demon cried blood , the blood fell on the bed , the bed is now possessed and it now eats . xxmaj along with fruit , flowers"
3,"evil xxmaj witch xxmaj doctor orders to execute him . xxmaj he then comes back as a murdering tree ( ! ) , xxmaj xxunk . xxmaj well , what can you say about such a "" film "" ? xxmaj if it was intended to be a horror film , there obviously was some sort of bad judgment involved . xxmaj and for a comedy , it still is"
4,"the top name in a b - movie , even in this case starring along side xxmaj charles xxmaj napier , xxmaj master xxmaj control xxmaj program , and xxmaj joe xxmaj don xxmaj baker . xxmaj as always he does a great job of being the bad guy , but the plot is just bad ( do n't even get me started about the ending ) . xxmaj and"


In [11]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)  #####create language model learner
 

learn.lr_find()

learn.recorder.plot()




epoch,train_loss,valid_loss,accuracy,time


LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


RuntimeError: CUDA out of memory. Tried to allocate 802.00 MiB (GPU 0; 4.00 GiB total capacity; 2.11 GiB already allocated; 501.90 MiB free; 2.17 GiB reserved in total by PyTorch)

In [ ]:
learn.fit_one_cycle(5, slice())

learn.save_encoder('fine-tuned-enc')

In [ ]:
data_clas = (TextList.from_folder(path, vocab=data_lm.vocab)
             #grab all the text files in path
             .split_by_folder(valid='test')
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             .label_from_folder(classes=['neg', 'pos'])
             #label them all with their folders
             .databunch(bs=50))


##this is for classify movie reviews-----------

learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.3) 
learn.load_encoder('fine-tuned-enc')

learn.lr_find()

learn.recorder.plot()


learn.fit_one_cycle(5, slice())


learn.predict("I really loved that movie, it was awesome!")
